# 🧪 3.4 – Laboratorio: Funciones y Decoradores Aplicados

En este laboratorio crearás un **sistema de registro y validación de operaciones matemáticas**, combinando:

✅ Funciones con argumentos variables (*args, **kwargs*)  
✅ Closures para almacenar estado  
✅ Decoradores para añadir validación, tiempo y registro

---
## 🎯 Objetivos
- Reforzar el uso de funciones con argumentos variables.
- Aplicar closures y decoradores en un caso práctico.
- Diseñar una API funcional reutilizable y extensible.
- Introducir el concepto de validación y logging automático de operaciones.

In [2]:
print('✅ Laboratorio 3.4 — Funciones y Decoradores listos para practicar.')

✅ Laboratorio 3.4 — Funciones y Decoradores listos para practicar.


---
## 1️⃣ Escenario

Queremos construir un pequeño **sistema de cálculo** que permita:

- Registrar operaciones matemáticas (suma, resta, multiplicación, división).
- Validar los argumentos antes de ejecutar.
- Medir el tiempo que tarda cada operación.
- Guardar un registro con el historial de operaciones.

Usaremos **decoradores** para añadir validación, registro y medición de tiempo.

---
## 2️⃣ Paso 1 – Closure para el historial de operaciones

Crea una función `crear_historial()` que devuelva dos funciones:

- `registrar(operacion)` → añade una operación al historial.
- `mostrar()` → devuelve la lista de operaciones.

💡 *Pista:* usa una lista local `historial = []` y la palabra clave `nonlocal`.

In [29]:
# TODO: implementa el closure del historial

def crear_historial():
    historial = []

    def registrar(operacion):
        nonlocal historial
        historial.append(operacion)

    def mostrar():
        return historial

    return registrar, mostrar

In [23]:
r1, m1 = crear_historial()

r1('hola mundo')
r1('adios mundo')

print(m1())

['hola mundo', 'adios mundo']


In [24]:
# Test
registrar, mostrar = crear_historial()
registrar('prueba')
assert 'prueba' in mostrar(), '❌ El historial no guarda correctamente'
print('✅ Closure funcional.')

✅ Closure funcional.


💡 **Tip:**
- Define `historial = []` dentro de `crear_historial()`.
- Crea dos funciones internas `registrar()` y `mostrar()`.
- Usa `nonlocal` si necesitas modificar la lista desde dentro.

✅ **Solución explicada:** el closure mantiene una lista privada accesible solo mediante las funciones internas.

---
## 3️⃣ Paso 2 – Decorador de validación de tipos

Crea un decorador `validar_numeros(func)` que asegure que **todos los argumentos posicionales** son numéricos.
Si alguno no lo es, lanza `TypeError('❌ Todos los argumentos deben ser numéricos')`.

In [66]:
# TODO: crea el decorador validar_numeros()
def validar_numeros(func):
    def validador(*args, **kwargs):
        if not all(isinstance(x, (int, float)) for x in args):
            raise TypeError('❌ Todos los argumentos deben ser numéricos')
        return func(*args, **kwargs)
    return validador


In [65]:
# Test


@validar_numeros
def suma_prueba(a, b):
    return a + b

assert suma_prueba(2, 3) == 5, '❌ No devuelve la suma esperada'
try:
    suma_prueba('a', 2)
except TypeError:
    print('✅ Validación de tipo correcta.')


✅ Validación de tipo correcta.


💡 **Tip:** dentro del decorador:
- Usa `all(isinstance(x, (int, float)) for x in args)`.
- Si hay un error, lanza `TypeError()`.

✅ **Solución explicada:** el decorador actúa como guardia de tipo antes de ejecutar la función real.

---
## 4️⃣ Paso 3 – Decorador de registro y tiempo de ejecución

Crea un decorador `registrar_tiempo(func)` que:
- Mida el tiempo con `time.time()`.
- Ejecute la función decorada.
- Guarde un mensaje en el historial (`registrar`).
- Devuelva el resultado original.

💡 *Pista:* usa variables externas (`registrar`) del closure.

In [41]:
# TODO: crea el decorador registrar_tiempo()

import time

# def registro():
#     registro = []
#     def registrar(mensaje):    
#         nonlocal registro
#         registro.append(mensaje)    
#     def leer():
#         return registro
#     return registrar, leer

# registrar, leer = registro()


def registrar_tiempo(register):
    def decorator(func):
        def wrapper(*args, **kwargs):
            inicio = time.time()
            resultado = func(*args, **kwargs)
            fin = time.time()
            duracion = round(fin - inicio, 5)
            mensaje = f"⏱️ {func.__name__}{args} = {resultado} ({duracion}s)"
            # if 'registrar' in globals():
            register(mensaje)
            return resultado
        return wrapper
    return decorator


In [67]:
# Test
# 
r1, m1 = crear_historial()
r2, m2 = crear_historial()

@registrar_tiempo(r1)
def sumar(a, b):
    return a + b

@registrar_tiempo(r2)
def restar(a, b):
    return a - b


restar(1, 2)
sumar(2,3)
assert len(m2()) >= 0, '❌ No se registró la operación'
print('✅ Decorador de registro funcional.')
print(m2())
print(m1())

✅ Decorador de registro funcional.
['⏱️ restar(1, 2) = -1 (0.0s)']
['⏱️ sumar(2, 3) = 5 (0.0s)']


💡 **Tip:**
- Mide `inicio = time.time()` y `fin = time.time()`.
- Crea un mensaje con `f"{func.__name__}{args} = {resultado}"`.
- Usa `registrar(mensaje)` para guardar el log.

✅ **Solución explicada:** el decorador añade comportamiento extra sin modificar la lógica principal.

---
## 5️⃣ Paso 4 – Funciones matemáticas decoradas

Crea funciones `sumar`, `restar`, `multiplicar`, `dividir` aplicando ambos decoradores.

💡 *Importante:* el orden de los decoradores es **primero validación**, luego registro.

In [70]:
# TODO: implementa las funciones decoradas

r3, m3 = crear_historial()

@validar_numeros
@registrar_tiempo(r3)
def sumar(a, b):
    return a + b

@validar_numeros
@registrar_tiempo(r3)

def restar(a, b):
    return a - b

@validar_numeros
@registrar_tiempo(r3)

def multiplicar(a, b):
    return a * b

@validar_numeros
@registrar_tiempo(r3)
def dividir(a, b):
    if b == 0:
        raise ZeroDivisionError("❌ No se puede dividir entre cero")
    return a / b


In [76]:
# Test
# registrar, mostrar = crear_historial()
# sumar(3,4)
# assert any('sumar' in op for op in m3()), '❌ No se registró la suma'
# print('✅ Decoradores aplicados correctamente.')


sumar(3,4)
multiplicar(4,4)
dividir(1,1)

m3()

['⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)']

💡 **Tip:**
- `@registrar_tiempo` debe estar **encima** de `@validar_numeros`.
- Usa `return a + b`, `return a - b`, etc.

✅ **Solución explicada:** el flujo de llamada pasa por validación, luego registro, y finalmente ejecuta la operación.

---
## 6️⃣ Reto final — Añadir una nueva operación

Crea una función `potencia(base, exponente)` decorada igual que las demás.
Ejecuta varias operaciones y muestra el historial final.

In [78]:
# TODO: crea la función potencia()
# potencia(2, 3)
# print(mostrar())


@validar_numeros
@registrar_tiempo(r3)
def potencia(base, exponente):
    return base ** exponente

potencia(2,2)
m3()

['⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)',
 '⏱️ sumar(3, 4) = 7 (0.0s)',
 '⏱️ multiplicar(4, 4) = 16 (0.0s)',
 '⏱️ dividir(1, 1) = 1.0 (0.0s)',
 '⏱️ potencia(2, 2) = 4 (0.0s)']

💡 **Tip:** usa `base ** exponente` dentro de la función.

✅ **Solución explicada:** el decorador registra automáticamente la llamada y su duración.

---
## 🧠 Resumen del laboratorio

- Has combinado **closures y decoradores** en un sistema completo.
- Has implementado validación, registro y medición automática.
- Este patrón es base de frameworks como **Flask**, **FastAPI** o **pytest**.